<a href="https://colab.research.google.com/github/urosgodnov/juypterNotebooks/blob/main/DataMining/text_mining_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df=pd.read_csv("Alien.csv")
df.drop('Unnamed: 0', axis=1,inplace=True)
df.head()

,Author,Title,Rating,Text,Date
0,kalle-parviainen,High-budget Alien fan film,7,A few hours after coming out of the theater an...,2024-08-14
1,andyelliott-53082,An Alien Greatest Hits album sung by a cover b...,6,"Positives first - Good action, great practical...",2024-08-17
2,bradaviel,The Rogue One of the Alien Franchise,8,I had high hopes for this one and what can I s...,2024-08-14
3,Pigeons1,"""In space no one can hear you sigh""",6,"At this point in 2024, there are more bad alie...",2024-08-15
4,pootc,All too familiar,6,Alien: Romulus delivers a gritty experience re...,2024-08-17


In [8]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
word_tokenize

**Retains punctuation as separate tokens.**

In [9]:
df['tokenized_sentences'] = df['Text'].astype(str).str.lower().apply(word_tokenize)

In [ ]:
df['tokenized_sentences'].head()

In [ ]:
len(df['tokenized_sentences'].iloc[0])
df['tokenized_sentences'].iloc[0]

In [ ]:
from nltk.tokenize import RegexpTokenizer

regexp = RegexpTokenizer('\w+')

df['text_token']=df['Text'].astype(str).str.lower().apply(regexp.tokenize)

df['text_token'].head()

**Punctuations are omitted**

In [19]:
len(df['text_token'].iloc[0])
df['text_token'].iloc[0]

300

In [31]:
df['text_token'] = df['text_token'].apply(lambda x: [item for item in x if not re.search(r'\d', item)])

In [ ]:
len(df['text_token'].iloc[0])
df['text_token'].iloc[0]

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

# Make a list of english stopwords
stopwords = nltk.corpus.stopwords.words("english")

# Extend the list with your own custom stopwords
my_stopwords = []
stopwords.extend(my_stopwords)

In [23]:
stopwords[0:5]

['i', 'me', 'my', 'myself', 'we']

In [29]:
import re

In [ ]:
df['text_token'] = df['text_token'].apply(lambda x: [item for item in x if item not in stopwords])

df['text_token'].head()

In [ ]:
len(df['text_token'].iloc[0])
df['text_token'].iloc[0]

In [ ]:
df['text_string'] = df['text_token'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

df[['Text','text_token','text_string']].head()

In [ ]:
from nltk.probability import FreqDist

all_words = ' '.join([word for word in df['text_string']])

tokenized_words = nltk.tokenize.word_tokenize(all_words)

fdist = FreqDist(tokenized_words)
fdist

In [ ]:
df['text_string_fdist'] = df['text_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 500 ]))

df['text_string_fdist'].head()

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

In [38]:
from nltk.stem import WordNetLemmatizer

wordnet_lem = WordNetLemmatizer()

df['text_string_lem'] = df['text_string_fdist'].apply(lambda x:' '.join([wordnet_lem.lemmatize(item) for item in x.split(' ')]))



In [ ]:
df[['Text', 'text_token', 'text_string', 'text_string_fdist','text_string_lem']].head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from wordcloud import WordCloud

all_words_lem = ' '.join([word for word in df['text_string_lem']])
wordcloud = WordCloud(width=600,
                     height=400,
                     random_state=2,
                     max_font_size=100).generate(all_words_lem)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')

In [41]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

words = nltk.word_tokenize(all_words_lem)
fd = FreqDist(words)

fd.tabulate(10)

    alien     movie      film      like franchise       one      good   romulus  original character 
     4512      3543      2525      1315      1070      1044       981       962       816       802 


In [ ]:
# Obtain top 10 words
top_20 = fd.most_common(20)

# Create pandas series to make plotting easier
fdist = pd.Series(dict(top_20))
import seaborn as sns
sns.set_theme(style="ticks")

sns.barplot(y=fdist.index, x=fdist.values, color='blue')

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

df['polarity'] = df['text_string_lem'].apply(lambda x: analyzer.polarity_scores(x))

df['polarity'].head(5)

In [ ]:
df = pd.concat(
    [df.drop(['polarity'], axis=1),
     df['polarity'].apply(pd.Series)], axis=1)
df.head(3)

In [ ]:
df['sentiment'] = df['compound'].apply(lambda x: 'positive' if x >0 else 'neutral' if x==0 else 'negative')

df[['Text','sentiment']].head(10)

In [ ]:
df.loc[df['compound'].idxmax(),"Text"]

In [ ]:
df.loc[df['compound'].idxmin(),["Text", "sentiment"]]

In [ ]:
sns.countplot(y='sentiment',
             data=df,
             palette=['#b2d8d8',"#008080", '#db3d13']
             );

In [ ]:
sns.boxplot(y='compound',
            x='sentiment',
            hue='sentiment',  # Assign 'sentiment' to hue
            palette=['#b2d8d8',"#008080", '#db3d13'],
            data=df,
            legend=False  # Set legend to False
            );

In [ ]:
finder = nltk.collocations.BigramCollocationFinder.from_words(words)
finder.ngram_fd.most_common(10)

In [ ]:
finder = nltk.collocations.TrigramCollocationFinder.from_words(words)
finder.ngram_fd.most_common(10)

In [55]:
import pandas as pd

data=pd.read_csv("Alien.csv")

data=data.drop(columns=['Unnamed: 0','Author', 'Title','Rating','Date'], axis=1)

In [ ]:
import re
# Remove punctuation
data['text_processed'] = \
data['Text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
data['text_processed'] = \
data['text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
data['text_processed'].head()

In [57]:
import gensim
from gensim.utils import simple_preprocess
import nltk

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend([])

In [58]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

In [59]:
data = data.text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)

In [60]:
print(data_words[:1][0][:30])

['hours', 'coming', 'theater', 'sure', 'make', 'okay', 'guessit', 'felt', 'like', 'fedy', 'alvarez', 'examined', 'production', 'previous', 'films', 'every', 'left', 'idea', 'thrown', 'finally', 'screen', 'along', 'rehashing', 'core', 'components', 'franchise', 'pieces', 'alien', 'aliens', 'resurrection']


In [61]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 2)]


In [63]:
from pprint import pprint
# number of topics
num_topics = 3
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.021*"alien" + 0.019*"movie" + 0.009*"film" + 0.007*"good" + 0.007*"franchise" + 0.006*"one" + 0.006*"movies" + 0.006*"like" + 0.005*"really" + 0.005*"original"'), (1, '0.026*"alien" + 0.015*"movie" + 0.011*"film" + 0.008*"like" + 0.008*"one" + 0.007*"romulus" + 0.007*"franchise" + 0.006*"characters" + 0.005*"really" + 0.005*"aliens"'), (2, '0.021*"alien" + 0.016*"movie" + 0.015*"film" + 0.011*"like" + 0.006*"good" + 0.006*"aliens" + 0.006*"original" + 0.006*"movies" + 0.006*"franchise" + 0.006*"first"')]


In [65]:
!pip install pyLDAvis --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.9 MB/s eta 0:00:00


In [ ]:
import pyLDAvis.gensim
import pickle
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_three_topics_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_three_topics_'+ str(num_topics) +'.html')
LDAvis_prepared